<a href="https://colab.research.google.com/github/thehapyone/Thesis_Project/blob/master/Extraction_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feature Engineering & Data Cleaning and Augumentation Complete Pipeline
This code performs different levels of data augumentation on the provided data automatically.

In [0]:
import pandas as pd
import numpy as np
import copy as copy
# using interativeimputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

## Duplicates Removal Pipeline
This stage removes duplicates features from the input data.
Input - A Pandas dataframe with n columns
Output - A Pandas Dataframe with n-d columns. Where d is the no of duplicates columns

In [0]:
# function returns the duplicate columns
def getDuplicateColumns(df):
    '''
    Get a list of duplicate columns.
    It will iterate over all the columns in dataframe and find the columns whose contents are duplicate.
    :param df: Dataframe object
    :return: List of columns whose contents are duplicates.
    '''
    duplicateColumnNames = set()
    # Iterate over all the columns in dataframe
    for x in range(df.shape[1]):
        # Select column at xth index.
        col = df.iloc[:, x]
        # Iterate over all the columns in DataFrame from (x+1)th index till end
        for y in range(x + 1, df.shape[1]):
            # Select column at yth index.
            otherCol = df.iloc[:, y]
            # Check if two columns at x 7 y index are equal
            if col.equals(otherCol):
                duplicateColumnNames.add(df.columns.values[y])
 
    return list(duplicateColumnNames)

# function removes duplicate features from the data
def removeDuplicates(raw_data, debug=False):
  # fetch out the duplicates columns - It will take sometime: Quadratic time complexity
  dup_columns = getDuplicateColumns(raw_data)
  if debug == True:
    print ('Duplicate columns are: ')
    print (dup_columns)

  # now we drop all the duplicate columns
  output = raw_data.drop(labels=dup_columns, axis=1)

  return output

# calling this is easy as
# my_data = removeDuplicates(raw_data)

## Remove Zero Variance Pipeline
This removes all feature with no changes to their elements


In [0]:
def removeZeroVariance(raw_data, debug=False):
  # finding column data with almost zero standard deviation. 
  std_data = raw_data.describe().transpose().loc[:,'std']

  # this columns have zero standard deviation
  zero_std_columns = std_data[std_data == 0]

  if debug == True:
    print ("Features with Zero Variance")
    print (zero_std_columns)

  # drop all features with zero variance
  zero_std_columns_list = list(zero_std_columns.index)
  output = raw_data.drop(labels=zero_std_columns_list, axis=1)

  return output

## Drop Redundant Features
Here we can remove features we think isn't going to be helpful.
The function will support user inputted features as well

In [0]:
def dropRedundantFeatures(raw_data, to_remove=[], debug=False):
  # we have define a set of features to be removed normally
  if len(to_remove) == 0:
    to_remove = ['Index', 'DELIVERY_DATE', 'LAST_RUN', 'LAST_RUN.1']
  
  # now we drop the columns
  output = raw_data.drop(labels=to_remove, axis=1)
  return output

## Date Information Extractor
Here, we will extact information about the day, month, and year from the BiWeek_Send_Date feature.

In [0]:
def dateExtractor(raw_data, debug=True):

  output = copy(raw_data)
  # function extracts out the day, month, and year information from the 'BiWeek Send data' information.
  output['BIWEEK_SEND_DATE'] = pd.to_datetime(raw_data['BIWEEK_SEND_DATE'], infer_datetime_format=True)

  # Now we will extract out the day, month, and year information from the 'BiWeek Send data' information.
  day_data = output['BIWEEK_SEND_DATE'].dt.day
  month_data = output['BIWEEK_SEND_DATE'].dt.month
  year_data = output['BIWEEK_SEND_DATE'].dt.year

  # insert into the dataframe
  # Re-order the arrangments of the columns. Goal is to move 'Day', 'Month', and 'year' to the front
  output.insert(1, 'DAY', day_data)
  output.insert(2, 'MONTH', month_data)
  output.insert(3, 'YEAR', year_data)

  # now we will drop the BIWEEK_SEND_DATE
  output = output.drop(labels=['BIWEEK_SEND_DATE'], axis=1)
  return output

## Filling Missing Values Pipeline
In this stage, I attempt to fill missing values in the data


*   The following features will be cleaned and updated - 'LX_PSC_P1FUO_NUMBER_OF_STARTER_ACTIVATION_D', 'LX_PSC_P1FUN_NUMBER_OF_ENGINE_STARTS_DURING', 'LX_PST_P1B3A_332_BCLVEHICLEMODECRANKINGTIME_LOG'
*   The following features will be cleaned and updated - LX_PSC_P1ASU_MAIN_LOG_DRIVE_FUEL and LX_PSC_P1ATK_MAIN_LOG_ECONOMICAL_FUEL



In [0]:
def stageFilling(raw_data, impute_cols=[], debug=False):
  output = copy.copy(raw_data)
  # create the imputer
  imp = IterativeImputer(max_iter=10, random_state=42)
  # stage 1 of the filling pipeline
  missing_cols_1 = impute_cols
  # create the dataFrame
  data_stage1 = output[missing_cols_1]

  # see the current state
  if debug == True:
    print ("Before imputation")
    print(data_stage1.isnull().sum().sort_values(ascending=False))
  
  # perform imputation on the missing values
  result = pd.DataFrame(imp.fit_transform(data_stage1.values), columns=data_stage1.columns)

  # see the resulting state
  if debug == True:
    print ("After imputation")
    print(data_stage1.isnull().sum().sort_values(ascending=False))

  # now update the dataframe with the new changes
  output.update(results)

  return output


def fillMissingValues(raw_data, debug=False):
  # We define the columns we want to perform imputation on
  missing_cols = ['LX_PSC_P1FUO_NUMBER_OF_STARTER_ACTIVATION_D', 'LX_PSC_P1FUN_NUMBER_OF_ENGINE_STARTS_DURING', 'LX_PST_P1B3A_332_BCLVEHICLEMODECRANKINGTIME_LOG']
  results1 = stageFilling(raw_data, impute_cols=missing_cols, debug=debug)

  # for stage 2
  # We define the columns we want to perform imputation on
  missing_cols = ['LX_PSC_P1ASU_MAIN_LOG_DRIVE_FUEL', 'LX_PSC_P1BBY_TOTAL_FUEL_CONSUMPTION', 'LX_PSC_P1ATO_MAIN_LOG_TOTAL_ENERGY', 'LX_PSC_P1ATK_MAIN_LOG_ECONOMICAL_FUEL']
  results2 = stageFilling(results1, impute_cols=missing_cols, debug=debug)

  # other high level imputation can be perfromed here as well
  
  return results2